R magic
=============

In [3]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R/tutorial/dataset/OLAP
!ls

/content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R/tutorial/dataset/OLAP
cheese_table.csv  location_table.csv  province_table.csv  supplier_table.csv
city_table.csv	  month_table.csv     size_table.csv	  topping_table.csv
dough_table.csv   order_table.csv     store_table.csv


Generate dimension tables and store in .csv
================================

In [5]:
%%R
cheese_table <- data.frame(key=1:3,
            name=c("Swiss", "Cheddar", "Mozzzarella"),
             supplier_key=1:3)
write.csv(cheese_table,
          "cheese_table.csv",
          row.names=F)
cheese_table

  key        name supplier_key
1   1       Swiss            1
2   2     Cheddar            2
3   3 Mozzzarella            3


In [70]:
%%R
dough_table <- data.frame(key=1:3,
            name=c("whole wheat thin", "white regular", "stuffed crust"))
write.csv(dough_table,
          "dough_table.csv",
          row.names=F)
dough_table

  key             name
1   1 whole wheat thin
2   2    white regular
3   3    stuffed crust


In [7]:
%%R
size_table <- data.frame(key=1:4,
             name=c("small", "medium", "large", "xlarge"),
             specification=c("8 inches", "12 inches", "14 inches", "16 inches"))
write.csv(size_table,
          "size_table.csv",
          row.names=F)
size_table

  key   name specification
1   1  small      8 inches
2   2 medium     12 inches
3   3  large     14 inches
4   4 xlarge     16 inches


In [8]:
%%R
supplier_table <- data.frame(key=1:3,
             name=c("Ubisoft", "EA", "KONAMI"))
write.csv(supplier_table,
          "supplier_table.csv",
          row.names=F)
supplier_table

  key    name
1   1 Ubisoft
2   2      EA
3   3  KONAMI


In [56]:
%%R
city_table <- data.frame(key=1:4,
             name=c("Ottawa", "Toronto", "Edmonton", "Calgary"),
             province_key=c(1, 1, 2, 2))
write.csv(city_table,
          "city_table.csv",
          row.names=F)
city_table

  key     name province_key
1   1   Ottawa            1
2   2  Toronto            1
3   3 Edmonton            2
4   4  Calgary            2


In [10]:
%%R
location_table <- data.frame(key=1:4,
             name=c("1123, Holland Street", "304, Baseline Street", "5463, Friday Street", "43, Somerset Street"),
             city_key=1:4)
write.csv(location_table,
          "location_table.csv",
          row.names=F)
location_table

  key                 name city_key
1   1 1123, Holland Street        1
2   2 304, Baseline Street        2
3   3  5463, Friday Street        3
4   4  43, Somerset Street        4


In [11]:
%%R
province_table <- data.frame(key=1:2,
             name=c("Ontario", "Alberta"),
             country=c("Canada", "Canada"))
write.csv(province_table,
          "province_table.csv",
          row.names=F)
province_table

  key    name country
1   1 Ontario  Canada
2   2 Alberta  Canada


In [12]:
%%R
store_table <- data.frame(key=1:4,
             location_key=1:4,
             manager=c("Mike", "Bob", "Mary", "Jessie"))
write.csv(store_table,
          "store_table.csv",
          row.names=F)
store_table

  key location_key manager
1   1            1    Mike
2   2            2     Bob
3   3            3    Mary
4   4            4  Jessie


In [13]:
%%R
topping_table <- data.frame(key=1:4,
             name=c("tomatoes", "pepper", "onions", "pepperoni"))
write.csv(topping_table,
          "topping_table.csv",
          row.names=F)
topping_table

  key      name
1   1  tomatoes
2   2    pepper
3   3    onions
4   4 pepperoni


In [14]:
%%R
month_table <- 
  data.frame(key=1:12,
            desc=c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"),
            quarter=c("Q1","Q1","Q1","Q2","Q2","Q2","Q3","Q3","Q3","Q4","Q4","Q4"))
write.csv(month_table,
          "month_table.csv",
          row.names=F)
month_table

   key desc quarter
1    1  Jan      Q1
2    2  Feb      Q1
3    3  Mar      Q1
4    4  Apr      Q2
5    5  May      Q2
6    6  Jun      Q2
7    7  Jul      Q3
8    8  Aug      Q3
9    9  Sep      Q3
10  10  Oct      Q4
11  11  Nov      Q4
12  12  Dec      Q4


generate fact table and store in .csv
============================

sample from all necessary dimension tables

In [71]:
%%R
sample_num = 100
store <- sample(store_table$key, sample_num, replace=T)
month <- sample(month_table$key, sample_num, replace=T)
cheese <- sample(cheese_table$key, sample_num, replace=T)
topping <- sample(topping_table$key, sample_num, replace=T)
size <- sample(size_table$key, sample_num, replace=T)
dough <- sample(dough_table$key, sample_num, replace=T)

the profit is determined by size, so we have

In [72]:
%%R
profit <- 1 * size

and we hope the amount of xlarge pizza will increase along with the increase of the month
$$ amount = month * size\_weights(size) + noise$$

In [73]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 0
   } else if(x == 3){
       w <- 1
   }
   else{
       w <- 3
   }
   w
}

weight <- sapply(size, size_weights)
print(head(size, 20))
print(head(weight, 20))
noise <- sample(1:10, sample_num, replace=T)
amount <- month*weight+noise
print(head(amount, 20))

 [1] 1 3 1 1 3 4 4 1 2 1 2 1 1 1 1 3 3 3 3 3
 [1] 0 1 0 0 1 3 3 0 0 0 0 0 0 0 0 1 1 1 1 1
 [1] 10 15 10  8  7 31 28  7  3  2  8  6  4  9  8  7 14 20  9 16


we assemble them together to get a function

In [114]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 0
   } else if(x == 3){
       w <- 1
   }
   else{
       w <- 3
   }
   w
}

generate_orders <- function(sample_num){
    store <- sample(store_table$key, sample_num, replace=T)
    month <- sample(month_table$key, sample_num, replace=T)
    cheese <- sample(cheese_table$key, sample_num, replace=T)
    topping <- sample(topping_table$key, sample_num, replace=T)
    size <- sample(size_table$key, sample_num, replace=T)
    dough <- sample(dough_table$key, sample_num, replace=T)

    profit <- 1 * size

    weight <- sapply(size, size_weights)
    noise <- sample(1:10, sample_num, replace=T)
    amount <- month*weight+noise

    # generate the fact table(order)
    order_table <- data.frame(order_key=1:sample_num,
                      store_key=store,
                      month_key=month,
                      size_key=size,
                      dough_key=dough,
                      cheese_key=cheese,
                      topping_key=topping,
                      amount=amount,
                      profit=profit)
    order_table <- order_table[order(order_table$store_key, -order_table$month_key),]
    row.names(order_table) <- NULL
    return(order_table)
}

new let's generate a fact table

In [123]:
%%R
order_table <- generate_orders(5000)
write.csv(order_table,
          "order_table.csv",
          row.names=F)
head(order_table,20)

   order_key store_key month_key size_key dough_key cheese_key topping_key
1          5         1        12        1         1          3           4
2        241         1        12        4         3          2           3
3        243         1        12        1         3          3           4
4        255         1        12        1         2          2           1
5        335         1        12        1         3          2           1
6        352         1        12        3         3          2           3
7        405         1        12        2         2          2           2
8        639         1        12        2         1          3           4
9        681         1        12        3         3          2           1
10       728         1        12        4         2          2           1
11       731         1        12        2         2          3           1
12       839         1        12        4         2          3           1
13       849         1   

read from .csv
==================================

In [125]:
%%R
month_table <- read.csv("month_table.csv")
topping_table <- read.csv("topping_table.csv")
store_table <- read.csv("store_table.csv")
province_table <- read.csv("province_table.csv")
location_table <- read.csv("location_table.csv")
city_table <- read.csv("city_table.csv")
supplier_table <- read.csv("supplier_table.csv")
size_table <- read.csv("size_table.csv")
dough_table <- read.csv("dough_table.csv")
cheese_table <- read.csv("cheese_table.csv")

order_table <- read.csv("order_table.csv")

check if the order table is correct

In [127]:
%%R
head(order_table)

  order_key store_key month_key size_key dough_key cheese_key topping_key
1         5         1        12        1         1          3           4
2       241         1        12        4         3          2           3
3       243         1        12        1         3          3           4
4       255         1        12        1         2          2           1
5       335         1        12        1         3          2           1
6       352         1        12        3         3          2           3
  amount profit
1      9      1
2     43      4
3      5      1
4      2      1
5      5      1
6     22      3


OLAP operation
=========================
create a view associating with all dimension tables

In [141]:
%%R
fs <- merge(x=order_table, y=store_table[,c("key", "location_key")], by.x="store_key", by.y="key", all.x=T) # fact left join store

In [148]:
%%R
## month
fs <- merge(x=order_table, y=month_table[,c("key", "desc", "quarter")], by.x="month_key", by.y="key", all.x=T) # fact left join store
names(fs)[names(fs) == "desc"] <- "month"

## location
fs <- merge(x=fs, y=store_table[,c("key", "location_key")], by.x="store_key", by.y="key", all.x=T) # fact left join store
fs <- merge(x=fs, y=location_table[, c("key", "name", "city_key")], by.x="location_key", by.y="key", all.x=T) # with location table
fs <- merge(x=fs, y=city_table[, c("key", "name", "province_key")], by.x="city_key", by.y="key", all.x=T) # with city table
fs <- merge(x=fs, y=province_table, by.x="province_key", by.y="key", all.x=T) # with province table
## rename
names(fs)[names(fs) == "name.x"] <- "street"
names(fs)[names(fs) == "name.y"] <- "city"
names(fs)[names(fs) == "name"] <- "province"

## size
fs <- merge(x=fs, y=size_table[,c("key", "name")], by.x="size_key", by.y="key", all.x=T)
## rename
names(fs)[names(fs) == "name"] <- "size"

# dough
fs <- merge(x=fs, y=dough_table[,c("key", "name")], by.x="dough_key", by.y="key", all.x=T) 
## rename
names(fs)[names(fs) == "name"] <- "dough"

# cheese
fs <- merge(x=fs, y=cheese_table[,c("key", "name", "supplier_key")], by.x="cheese_key", by.y="key", all.x=T) 
fs <- merge(x=fs, y=supplier_table[,c("key", "name")], by.x="supplier_key", by.y="key", all.x=T)
## rename
names(fs)[names(fs) == "name.x"] <- "cheese"
names(fs)[names(fs) == "name.y"] <- "cheese_supplier"

# topping
fs <- merge(x=fs, y=topping_table[,c("key", "name")], by.x="topping_key", by.y="key", all.x=T) 
names(fs)[names(fs) == "name"] <- "topping"

# select
fs <- fs[c("order_key", "store_key", "month_key", "quarter", "city", "province", "country", "size", "dough", "cheese", "topping", "amount", "profit")]

head(fs)

  order_key store_key month_key quarter     city province country   size
1        81         4         9      Q3  Calgary  Alberta  Canada xlarge
2      2542         3        10      Q4 Edmonton  Alberta  Canada  large
3      3511         2        12      Q4  Toronto  Ontario  Canada medium
4       267         2         1      Q1  Toronto  Ontario  Canada xlarge
5      3742         1         1      Q1   Ottawa  Ontario  Canada  small
6      2468         3        12      Q4 Edmonton  Alberta  Canada medium
          dough      cheese  topping amount profit
1 white regular Mozzzarella tomatoes     31      4
2 stuffed crust       Swiss tomatoes     14      3
3 white regular       Swiss tomatoes      2      2
4 white regular       Swiss tomatoes      9      4
5 stuffed crust       Swiss tomatoes      1      1
6 white regular       Swiss tomatoes      8      2


build the OLAP cube along under store, month and size

In [168]:
%%R
# dimension <- c("store_key", "month_key", "size_key", "dough_key", "cheese_key", "topping_key")
dimension <- c("store_key", "month_key", "quarter", "size", "dough", "cheese", "topping", "province", "country")
revenue_cube <- 
    tapply(fs$amount, 
           fs[,dimension], 
           FUN=function(x){return(sum(x))})


show the cube of all dimension

In [150]:
%%R
revenue_cube

, , size = large, dough = stuffed crust, cheese = Cheddar, topping = onions

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1  7 NA NA  6  9 NA NA 24 15 13 NA 22
        2 NA 11 NA NA NA 14 NA NA NA 34 18 NA
        3 NA 13 NA  7  9 NA 18 NA 27 NA NA NA
        4 NA NA NA NA NA 19 16 11 NA NA 19 NA

, , size = medium, dough = stuffed crust, cheese = Cheddar, topping = onions

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 10  5  6  4  4  7 NA NA  5 NA 14 11
        2  4 NA NA  3  1 NA  7  6 10  7  6 NA
        3 17 NA NA  7 NA  1  5 NA 11 NA NA NA
        4  2 NA NA NA  4  2  9  6 NA NA NA  8

, , size = small, dough = stuffed crust, cheese = Cheddar, topping = onions

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 NA  4  2  7  2  7 NA NA  7 10 NA NA
        2  5  9 13 NA  4 11  6  9 NA  8 17 NA
        3 NA NA 24  7  8  4 10 NA NA  7 NA NA
        4 NA NA NA  5 NA 10  5 NA 10  1 NA 12

, , size = xlarge,

check the dimension name of the cube

In [151]:
%%R
dimnames(revenue_cube)

$store_key
[1] "1" "2" "3" "4"

$month_key
 [1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12"

$size
[1] "large"  "medium" "small"  "xlarge"

$dough
[1] "stuffed crust"    "white regular"    "whole wheat thin"

$cheese
[1] "Cheddar"     "Mozzzarella" "Swiss"      

$topping
[1] "onions"    "pepper"    "pepperoni" "tomatoes" 



roll up and drill down
===========================
month and size

In [169]:
%%R
apply(revenue_cube, c("month_key", "size"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

         size
month_key large medium small xlarge
       1    749    462   454    867
       2    740    504   655   1110
       3    759    523   627   1680
       4   1006    488   535   1940
       5    981    523   466   1973
       6   1160    561   595   2608
       7   1296    516   593   3333
       8   1420    545   593   3157
       9   1544    673   599   3662
       10  1744    550   530   3689
       11  1387    570   570   4191
       12  1797    573   601   3774


drill down: month, size and store

In [170]:
%%R
apply(revenue_cube, c("month_key", "size", "store_key"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

, , store_key = 1

         size
month_key large medium small xlarge
       1    303    126    73    224
       2    220    127   141    288
       3    173    199   178    332
       4    341    163   112    459
       5    262    158   111    534
       6    302    126   160    604
       7    386    138    97   1001
       8    497    123   196    824
       9    272    154   151    956
       10   499    129   144   1032
       11   431    108   127    881
       12   511    106   170    793

, , store_key = 2

         size
month_key large medium small xlarge
       1    121    130   156    214
       2    126    102   155    311
       3    195    158   144    400
       4    254    144   126    533
       5    242    164   102    433
       6    381    114   129    595
       7    250    116   173    989
       8    357    138   169    705
       9    404    189   160   1070
       10   469    117   134    509
       11   296    149   200   1167
       12   448    146   163    9

roll up, check the trend of size along with month and province

In [171]:
%%R
apply(revenue_cube, c("month_key", "size", "province"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

, , province = Alberta

         size
month_key large medium small xlarge
       1    325    206   225    429
       2    394    275   359    511
       3    391    166   305    948
       4    411    181   297    948
       5    477    201   253   1006
       6    477    321   306   1409
       7    660    262   323   1343
       8    566    284   228   1628
       9    868    330   288   1636
       10   776    304   252   2148
       11   660    313   243   2143
       12   838    321   268   1985

, , province = Ontario

         size
month_key large medium small xlarge
       1    424    256   229    438
       2    346    229   296    599
       3    368    357   322    732
       4    595    307   238    992
       5    504    322   213    967
       6    683    240   289   1199
       7    636    254   270   1990
       8    854    261   365   1529
       9    676    343   311   2026
       10   968    246   278   1541
       11   727    257   327   2048
       12   959    252 

roll up, check the trend of size along with month and province

In [172]:
%%R
apply(revenue_cube, c("quarter", "size", "province"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

, , province = Alberta

       size
quarter large medium small xlarge
     Q1  1110    647   889   1888
     Q2  1365    703   856   3363
     Q3  2094    876   839   4607
     Q4  2274    938   763   6276

, , province = Ontario

       size
quarter large medium small xlarge
     Q1  1138    842   847   1769
     Q2  1782    869   740   3158
     Q3  2166    858   946   5545
     Q4  2654    755   938   5378

